In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
from glob import glob
import upsilon
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit, least_squares
from statistics import mean, median
import pandas as pd
from dl import queryClient as qc
import collections
from astropy import stats
import os
import utils

-------------------------------
* WARNING: No pyFFTW detected *
-------------------------------


In [3]:
if not os.path.exists('results2'):
    os.makedirs('results2')
if not os.path.exists('results2/plots'):
    os.makedirs('results2/plots')

In [4]:
interesting = []
for obj in glob('interesting\*'):
    interesting.append(obj[12:-15])
interesting = list(set(interesting))

In [5]:
def get_data(df,objname):
    order = ['u','g','r','i','z']
    best_periods = []
    crv=[]
    fltrs=[]
    for f in order:
        selfltr = (df['filter'] == f)
        selfwhm = (df['fwhm'] <= 4.0)
        sel = selfltr & selfwhm
        t = df['mjd'][sel].values
        y = df['mag_auto'][sel].values
        dy = df['magerr_auto'][sel].values
        if len(t) < 25:
            continue

        best_periods.append(get_ls_period(t,y,objname=objname))
        crvi = np.vstack((t,y,dy)).T
        crv.append(crvi[np.argsort(crvi[:,0])])
        fltrs.append(f)
    period = 0
    for p in best_periods:
        period += p/len(best_periods)
    return crv, period, fltrs

In [6]:
def get_data(objname):
    df=qc.query(sql="""SELECT meas.* 
                     FROM nsc_dr2.meas
                     WHERE objectid='{:s}'""".format(objname),
              fmt='pandas',
              profile='db01')
    print(collections.Counter(df['filter']))
    order = ['u','g','r','i','z']
    best_periods = []
    crv=[]
    fltrs=[]
    for f in order:
        selfltr = (df['filter'] == f)
        selfwhm = (df['fwhm'] <= 4.0)
        sel = selfltr & selfwhm
        t = df['mjd'][sel].values
        y = df['mag_auto'][sel].values
        dy = df['magerr_auto'][sel].values
        if len(t) < 25:
            continue

        best_periods.append(get_ls_period(t,y,objname=objname,f=f))
        crvi = np.vstack((t,y,dy)).T
        crv.append(crvi[np.argsort(crvi[:,0])])
        fltrs.append(f)
    period = 0
    for p in best_periods:
        period += p/len(best_periods)
    return crv, period, fltrs
        
def get_ls_period(t,y,min_freq=1./1.,max_freq=1./0.1,objname='_',f=''):
    """Use Lomb-Scargle periodogram to get an estimate on period"""
    
    ls = stats.LombScargle(t, y)
    frequency, power = ls.autopower(minimum_frequency=min_freq,maximum_frequency=max_freq)
    period = 1./frequency # period is the inverse of frequency
    
    best_period = period[np.argmax(power)]
    
    plot_periodogram(period,power,best_period,objname=objname,f=f)
    return best_period

def plot_periodogram(period,power,best_period=None,objname='',ax=None,f=''):
   
    if ax is None:
        fig, ax = plt.subplots(figsize=(10,7))
        
    ax.plot(period,power,lw=0.1)
    ax.set_xlabel('period (days)')
    ax.set_ylabel('relative power')
    ax.set_title(objname)
    
    if best_period is not None:
        ax.axvline(best_period,color='r');
        ax.text(0.03,0.93,'period = {:.3f} days'.format(best_period),transform=ax.transAxes,color='r')
    fig.savefig('results2/plots/{}_{}_periodogram.pdf'.format(objname,f))
    plt.close(fig)
    return

def get_tmps(fltrs):
    tmps=[]
    typs =[]
    names=[]
    for fltr in fltrs:
        typ = []
        templets = glob('templets/*{}.dat'.format(fltr))
        tmp = np.zeros((len(templets),501,2))
        for i in range(len(templets)):
            tmp[i] = np.concatenate((np.array([[0,0]]),
                                     np.array(pd.read_csv(templets[i],sep=' ')),
                                     np.array([[1,0]])))
            #adjust if filepath to templets changes
            if len(templets[i])==17:
                typ.append('RRab')
            elif len(templets[i])==15:
                typ.append('RRc')
        typs.append(typ)
        names.append(templets)
        tmps.append(tmp)
    return tmps, names, typs

def double_tmps(tmps):
    tmps2=[]
    for f in range(len(tmps)):
        tmps2.append(np.tile(tmps[f],(2,1)))
        tmps2[f][:,int(len(tmps2[f][0])/2):,0] += 1
    return tmps2

def get_pinit(crv,period):
    pinit = ()
    for ltcrv in crv:
        pinit += ((0.0,max(ltcrv[:,1])-min(ltcrv[:,1]),0.0),)
    pinit += (period,)
    return pinit

def update_pinit(pars,period):
    pinit = ()
    for i in range(len(pars)):
        pinit += (tuple(pars[i,:-1]),)
    pinit += (period,)
    return pinit

def RemoveOutliers(crv,tmps,pars,period):
    n = pars[:,-1].astype(int)
    crv_in = []
    for i in range(len(crv)):
        f = interp1d(tmps[i][n[i],:,0],tmps[i][n[i],:,1]*pars[i,1]+pars[i,2])
        phase = (crv[i][:,0]/period-pars[i,0]) %1
        dif = abs(crv[i][:,1]-f(phase))
        crv_in.append(crv[i][dif<utils.mad(dif)*5])
    return crv_in

def double_period(crv,pars,period):
    crv2 = []
    for i in range(len(crv)):
        crv2.append(crv[i].copy())
        crv2[i][:,1] -= pars[i,2]
        
        crv2[i][:,0] = (crv2[i][:,0]/period-pars[i,0])%1
        crv2[i] = np.tile(crv2[i].T,2).T
        crv2[i][int(len(crv2[i])/2):,0] += 1
        crv2[i] = crv2[i][crv2[i][:,0].argsort()]
        
    return crv2

In [7]:
class tmpfitter:
    def __init__ (self, tmps):
        self.fltr=0
        self.n=0
        self.tmps=tmps

    def model(self, t, t0, amplitude, yoffset):
        # modify the template using peak-to-peak amplitude, yoffset
        # fold input times t by period, phase shift to match template
        xtemp = self.tmps[self.fltr][self.n,:,0]
        ytemp = self.tmps[self.fltr][self.n,:,1]*amplitude + yoffset
        ph = (t - t0) %1
        #print((ph[0],period,t0%1))
        #print((period,t0,amplitude,yoffset))
        # interpolate the modified template to the phase we want
        return interp1d(xtemp,ytemp)(ph)


def tmpfit(crv,tmps,pinit,w=.1,steps=21,n=1):
    fitter = tmpfitter(tmps)
    
    lsteps = int(steps/2+.5)
    rsteps = steps - lsteps
    pl = np.linspace(pinit[-1]-w,pinit[-1],lsteps)
    pr = np.linspace(pinit[-1]+w,pinit[-1],rsteps,endpoint=False)
    plist = np.zeros(pl.size+pr.size)
    plist[0::2] = np.flip(pl)
    plist[1::2] = np.flip(pr)
    plist = plist[plist>0]
    
    pars = np.zeros((len(tmps),4))
    minsumx2 = 10**50
    minp = 0
    for p in plist:
        sumx2=0
        ppars=np.zeros((len(tmps),4))
        for f in range(len(tmps)):
            fitter.fltr = f
            phase = crv[f][:,0]/p%n #1 for one period, 2 for two periods
            minx2 = 10**50
            for i in range(len(tmps[f])):
                fitter.n = i
                try:
                    tpars, cov = curve_fit(fitter.model, phase, crv[f][:,1], 
                                          bounds = ((-.5,0,-50),(.5,10,50)),
                                          sigma=crv[f][:,2], p0=pinit[f], maxfev=500)
                except RuntimeError:
                    #print('Error: Curve_fit failed on templet={}-{}, p={:.4}'.format(f,i,p))
                    continue
                
                x2 = sum((fitter.model(phase,tpars[0],tpars[1],tpars[2])-crv[f][:,1])**2/crv[f][:,2]**2)
                if x2 < minx2:
                    ppars[f,:-1] = tpars
                    ppars[f,-1] = i
                    minx2 = x2
            
            sumx2 += minx2
            if sumx2 > minsumx2:
                break
        if sumx2 < minsumx2:
            minsumx2 = sumx2
            minp = p
            pars = ppars
    npoints=0
    for i in range(len(crv)):
        npoints += len(crv[i])
    return pars, minp, minsumx2/npoints

In [ ]:
for objname in interesting:
    file = open("results2/{}_param.csv".format(objname),'a')
    print(objname)
    crv,p_est,fltrs = get_data(objname)
    if len(fltrs) == 0:
        continue
    tmps, tmpnames, typs = get_tmps(fltrs)
    
    pinit = get_pinit(crv,p_est)
    pars, p, x2 = tmpfit(crv,tmps,pinit,w=.1,steps=25)
    crv_in = RemoveOutliers(crv,tmps,pars,p)
    pinit = update_pinit(pars,p)
    pars_in,p_in,x2 = tmpfit(crv_in,tmps, pinit,w=.01,steps=25)
    
    crv2 = double_period(crv,pars_in,p_in)
    tmps2= double_tmps(tmps)
    n = pars[:,-1].astype(int)
    
    colors = []
    for f in fltrs:
        if f == 'r' or f == 'g':
            colors.append(f)
        else:
            colors.append('black')

    #Check if each filter is consistent with RR type (RRab or RRc)
    consistent = True
    for i in range(len(typs)):
        for j in range(i+1,len(typs)):
            if typs[i][n[i]] != typs[j][n[j]]:
                consistent = False
                break
        if not consistent:
            break
    if consistent:
        typ = typs[0][n[0]]
    else:
        typ = '???'
    fig, ax = plt.subplots(len(fltrs), figsize=(10,7.5), sharex=True, sharey=True)
    if len(fltrs) == 1:
        ax = [ax]
        
    file.write("{},X2:{:.3f},p:{:.3f}\n".format(objname,x2,p_in))
    for i in range(len(fltrs)):
        crvmean = mean(crv2[i][:,1])
        ax[i].scatter(crv2[i][:,0],crv2[i][:,1]-crvmean,c=colors[i])
        ax[i].plot(tmps2[i][n[i],:,0],tmps2[i][n[i],:,1]*pars_in[i,1]-crvmean,c='black')
        ax[i].invert_yaxis()
        ax[i].set_ylabel(fltrs[i], fontsize=18)
        
        data = crv2[i][:int(len(crv2[i])/2)]*np.array([p_in,1,1])
        rf_model = upsilon.load_rf_model()
        efeat = upsilon.ExtractFeatures(data[:,0],data[:,1],data[:,2])
        efeat.run()
        feats = efeat.get_features()
        rf_model = upsilon.load_rf_model()
        label, probability, flag = upsilon.predict(rf_model, feats)
        file.write('{},{},prob:{},flag:{},ndat:{}\n'.format(fltrs[i],label, probability, flag, len(data)))
        file.write("ph:{:.3f},amp:{:.3f},y0:{:.3f},{}\n".format(pars_in[i][0],pars_in[i][1]/2,pars_in[i][2],tmpnames[i][n[i]][9:]))
        np.savetxt('results2/{}_{}_data.csv'.format(objname,fltrs[i]),data,delimiter=',')

    ax[-1].set_xlabel('Phase', fontsize=16)
    ax[0].set_title("Object: {}    Period: {:.3f} d    Type: {}".format(objname,p_in,typ), fontsize=20)
    fig.savefig('results2/{}_lightcurves.pdf'.format(objname))
    
    file.write("---\n")
    plt.close(fig)
    file.close()

149139_18789
Counter({'g': 118, 'r': 110, 'z': 44, 'i': 41, 'VR': 11, 'Y': 5, 'u': 5})


C:\Users\kylem\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
C:\Users\kylem\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True,

147050_17040
Counter({'r': 36, 'g': 34, 'i': 33, 'z': 33, 'Y': 4})


C:\Users\kylem\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
C:\Users\kylem\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for 

148631_77026
Counter({'g': 89, 'r': 87, 'i': 4, 'VR': 4})


...

149142_141382
Counter({'g': 88, 'r': 87, 'VR': 7, 'i': 3, 'z': 1})


C:\Users\kylem\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
C:\Users\kylem\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for 

149139_84359
Counter({'g': 127, 'r': 126, 'z': 38, 'i': 35, 'VR': 10})


C:\Users\kylem\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int)
C:\Users\kylem\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for 

148120_6367
Counter({'g': 90, 'r': 87, 'i': 4, 'VR': 4})
